In [ ]:
from face_detector_image_generator import face_detection_and_capture
from nan_finder import find_nan_columns
from keypoints_display_df import show_multiple_images_with_keypoints_in_dataframe
from keypoints_display_tf import show_multiple_images_with_keypoints_in_tf_datasets
from keypoints_flip_tf import flip_coordinate_and_image_horizontal_in_tf_dataset, flip_coordinate_and_image_vertical_in_tf_dataset
from keypoints_flip_df import flip_coordinate_and_image_horizontal_in_df, flip_coordinate_and_image_vertical_in_df
from sobel_module import tf_compute_sobel
from canny_module import tf_compute_canny

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2

## **Deteksi Titik Kunci pada Wajah Menggunakan Jaringan Saraf Tiruan**
---

Setelah kita belajar mendeteksi wajah menggunakan metode Viola-Jones, langkah berikutnya adalah mendeteksi titik-titik kunci pada wajah menggunakan jaringan saraf tiruan (neural network). Titik-titik kunci ini bisa mencakup posisi mata, hidung, mulut, dan sebagainya.

#### Apa Itu Deteksi Titik Kunci Wajah?

**Deteksi titik kunci wajah** adalah proses menemukan titik-titik spesifik pada wajah, seperti ujung mata, sudut bibir, atau ujung hidung. Ini lebih rumit dibandingkan deteksi objek biasa karena kita harus menemukan banyak titik pada berbagai posisi dan orientasi di wajah, bukan hanya empat sudut dari kotak pembatas.

#### Langkah-langkah dalam Deteksi Titik Kunci

1. **Deteksi Wajah**: Pertama-tama, kita harus mendeteksi wajah dalam gambar, biasanya dengan menempatkannya dalam sebuah kotak pembatas (bounding box).
  
2. **Deteksi Titik Kunci**: Setelah wajah ditemukan, kita menggunakan jaringan saraf tiruan untuk mendeteksi titik-titik kunci pada wajah tersebut. Ini berarti kita harus menemukan lebih dari 10 titik berbeda yang merepresentasikan bagian-bagian penting dari wajah.

#### Menggunakan Data untuk Pelatihan

Untuk melatih jaringan saraf tiruan agar bisa mendeteksi titik kunci dengan baik, kita memerlukan banyak data. Salah satu sumber data yang bagus adalah dari Kaggle, di mana terdapat banyak gambar wajah beserta titik-titik kuncinya. Misalnya, tantangan deteksi titik kunci wajah di Kaggle menyediakan file CSV yang berisi link ke 7.049 gambar (berukuran 96 x 96 piksel), masing-masing dengan 15 titik kunci.

#### Proses Pelatihan

Untuk melatih model deteksi titik kunci:
1. **Kumpulkan Data**: Kumpulkan gambar wajah dan tandai titik-titik kuncinya.
2. **Persiapkan Data**: Format data tersebut agar bisa digunakan oleh jaringan saraf tiruan.
3. **Latih Model**: Gunakan data tersebut untuk melatih model jaringan saraf tiruan agar dapat mendeteksi titik-titik kunci pada wajah baru.

#### Mengapa Ini Penting?

Deteksi titik kunci pada wajah sangat berguna dalam berbagai aplikasi, seperti:
- **Pengenalan Wajah**: Membantu komputer mengenali siapa orang dalam gambar.
- **Animasi Wajah**: Digunakan dalam pembuatan karakter animasi yang realistis.
- **Analisis Ekspresi Wajah**: Membantu memahami emosi seseorang dari ekspresi wajahnya.

#### Kesimpulan

Deteksi titik kunci pada wajah adalah langkah lanjutan setelah deteksi wajah yang memungkinkan kita untuk menganalisis wajah dengan lebih detail. Dengan menggunakan jaringan saraf tiruan, kita bisa mendeteksi titik-titik kunci ini secara akurat, meskipun ini adalah tugas yang lebih rumit dibandingkan deteksi objek biasa.

Semoga penjelasan ini membantu! Jika ada yang masih membingungkan atau ada yang ingin ditanyakan lebih lanjut, jangan ragu untuk bertanya.

### **Mempersiapkan Dataset untuk Deteksi Titik Kunci Wajah**
---

Untuk melatih komputer agar bisa mendeteksi titik kunci pada wajah, kita perlu mengumpulkan dan mempersiapkan dataset yang berisi banyak gambar wajah. Berikut adalah langkah-langkah untuk membuat dataset ini menggunakan webcam di komputer kamu.

### Langkah-langkah Membuat Dataset

1. **Memuat Haar Cascade Classifier**
   - Pertama, kita perlu memuat classifier Haar cascade yang akan kita gunakan untuk mendeteksi wajah. Classifier ini adalah file XML yang bisa kamu temukan di folder OpenCV kamu.
   - Contoh kode:
     ```python
     face_cascade = cv2.CascadeClassifier('path_to_haarcascade_frontalface_default.xml')
     ```

2. **Mengatur Kamera**
   - Kita menggunakan webcam untuk mengambil gambar wajah. Perintah `cv2.VideoCapture(0)` digunakan untuk mengakses kamera internal komputer. Jika kamu menggunakan kamera eksternal, gunakan `cv2.VideoCapture(1)`.
   - Contoh kode:
     ```python
     cam = cv2.VideoCapture(0)
     ```

3. **Mendeteksi Wajah dan Menampilkan di Layar**
   - Setiap frame yang dibaca dari kamera diproses untuk mendeteksi wajah menggunakan classifier yang kita muat di langkah 1. Jika wajah terdeteksi, kotak pembatas akan digambar di sekitar wajah dan hanya bagian wajah yang akan ditampilkan di layar.
   - Contoh kode:
     ```python
     while(True):
         ret, frame = cam.read()
         faces = face_cascade.detectMultiScale(frame, 1.3, 5)
         for (x, y, w, h) in faces:
             if w > 130:
                 detected_face = frame[int(y):int(y+h), int(x):int(x+w)]
                 cv2.imshow("test", detected_face)
         if not ret:
             break
         k = cv2.waitKey(1)
     ```

4. **Menyimpan Gambar**
   - Gambar wajah yang terdeteksi kemudian diubah ukurannya menjadi 299x299 piksel dan disimpan dalam folder yang sudah kamu buat (misalnya, folder `dataset`). Setiap kali kamu menekan tombol spasi, gambar akan disimpan dengan nomor file yang terus bertambah.
   - Contoh kode:
     ```python
     faceresize = cv2.resize(detected_face, (299, 299))
     img_name = "dataset/opencv_frame_{}.jpg".format(img_counter)
     cv2.imwrite(img_name, faceresize)
     ```

5. **Mengumpulkan dan Mengannotasi Gambar**
   - Ambil sekitar 100 gambar wajah dengan berbagai posisi dan orientasi. Lebih banyak gambar akan memberikan hasil deteksi yang lebih baik. Untuk anotasi (penandaan) titik kunci pada wajah, kamu bisa menggunakan alat anotasi seperti VGG annotator.
   - VGG annotator memungkinkan kamu untuk menandai titik-titik kunci seperti mata, hidung, dan bibir pada gambar. Misalnya, kamu bisa menandai 16 titik pada wajah yang mencakup mata kiri, mata kanan, hidung, bibir, dan bentuk wajah luar.

### Kesimpulan

Dengan mengikuti langkah-langkah ini, kamu bisa membuat dataset sendiri yang berisi gambar-gambar wajah dengan titik-titik kunci yang telah dianotasi. Dataset ini sangat penting untuk melatih model jaringan saraf tiruan agar bisa mendeteksi titik-titik kunci pada wajah dengan akurat. Proses ini memerlukan kerja keras dan ketelitian, tetapi hasilnya akan sangat bermanfaat untuk berbagai aplikasi seperti pengenalan wajah dan analisis ekspresi wajah.

In [ ]:
# face_cascade_path = './Assets/data/haarcascades/haarcascade_frontalface_default.xml'
# face_detection_and_capture(face_cascade_path=face_cascade_path, output_dir='./Assets/Images/', img_size=(299, 299), alpha=0.1, offset=50)

### **Memproses Data Titik Kunci**
---

Ketika kita ingin mendeteksi titik-titik penting di wajah (seperti mata, hidung, dan mulut), kita membutuhkan data yang sudah di-annotate. Ada alat yang disebut VGG annotator yang bisa membantu kita dengan ini. Alat ini akan menghasilkan file CSV yang berisi koordinat (x, y) untuk setiap titik kunci pada setiap gambar.

#### Mengapa Ini Penting?

1. **Efisiensi**: Python kita tidak akan mencari banyak file gambar secara langsung, tetapi akan mencari data dari file CSV.
2. **Pengolahan Data**: Untuk setiap file CSV, ada 16 titik kunci yang harus diproses.
3. **Alternatif**: Ini adalah alternatif untuk menggunakan metode `ImageDataGenerator` dari Keras yang biasa digunakan untuk mengelola banyak file dalam direktori.

#### Langkah-Langkahnya

Untuk memudahkan pemahaman, kita akan membagi proses ini menjadi dua bagian:

1. **Praproses sebelum input ke dalam kode Keras-Python**
2. **Praproses dalam kode Keras-Python**

#### Praproses Sebelum Input ke Keras-Python

Sebelum kita memasukkan data ke dalam model Keras, kita perlu membersihkan dan menyiapkan data dari file CSV yang dihasilkan oleh VGG annotator. Proses ini melibatkan membaca file CSV, mengekstrak koordinat titik kunci, dan memastikan bahwa data siap untuk digunakan oleh model.

#### Praproses Dalam Kode Keras-Python

Setelah data siap, kita akan menggunakan Keras (sebuah pustaka untuk membuat dan melatih model pembelajaran mendalam) untuk memproses data lebih lanjut. Di sini, kita akan memastikan bahwa data diolah dengan benar saat dilatih oleh model.

#### Kesimpulan

Memproses data titik kunci memerlukan dua langkah utama: praproses sebelum memasukkan data ke dalam model Keras, dan praproses dalam model Keras itu sendiri. Dengan cara ini, kita memastikan bahwa data yang kita gunakan bersih, terstruktur, dan siap digunakan untuk melatih model pembelajaran mendalam yang akurat dan efisien.

In [ ]:
train_path = './Assets/datasets/training.csv'
test_path = './Assets/datasets/test.csv'

# train_path = './Assets/trainimgface.csv'
# test_path = './Assets/testimgface.csv'
train_data = pd.read_csv(train_path)  
test_data = pd.read_csv(test_path)

In [ ]:
train_data[:1]

In [ ]:
test_data[:1]

In [ ]:
train_data.info(memory_usage='deep')

In [ ]:
test_data.info(memory_usage='deep')

In [ ]:
get_nan_in_train_data = find_nan_columns(train_data, name_columns='facial_key_points', max_display=2)

In [ ]:
keypoints_columns = train_data.columns[:-1]
show_multiple_images_with_keypoints_in_dataframe(
    train_data, 
    image_column='Image',
    # parent_path='./Assets/Images/',
    keypoints_columns=keypoints_columns, 
    num_images=6, 
    image_size=(96, 96),
    figsize_per_image=(2, 2),
    scatter_size=100,
    n_seed=None
)

In [ ]:
train_data_copy = train_data.copy(deep=True)

In [ ]:
# train_data_copy.dropna(inplace=True)

In [ ]:
get_nan_in_train_data_copy = find_nan_columns(train_data_copy, name_columns='facial_key_points', max_display=3)

In [ ]:
print(f'''
data: {train_data_copy['Image'][0][:10]}
len: {len(train_data_copy['Image'][0][:10])}
type: {type(train_data_copy['Image'][0][:10])}
''')


In [ ]:
length_data = len(train_data_copy['Image'][[0]].str.split(' ')[0])
target_reshape = int(np.sqrt(length_data))

print(f'''
length data: {length_data}
target_reshape: {target_reshape}
''')

In [ ]:
train_data_images=train_data_copy['Image'].values
train_data_facial_keypoints=train_data_copy.drop(columns=['Image']).values

In [ ]:
facial_keypoints_train_datasets = tf.data.Dataset.from_tensor_slices((train_data_images, train_data_facial_keypoints))

print(f'info data: {facial_keypoints_train_datasets}')
print(f'number of data: {len(facial_keypoints_train_datasets)}')

In [ ]:
for img, keypoint in facial_keypoints_train_datasets.take(1).as_numpy_iterator():
    print(f"{'Check data'.center(61, '=')}")
    print(f'''    img value: {img[:20]}
    dtype img: {type(img)}
    keypoint value: {keypoint}
    dtype keypoint: {keypoint.dtype}'''
    )

    print(f"{'Convert to List'.center(61, '=')}")
    cvt_to_list = tf.strings.split(img, sep=' ')
    print(f'''    vector of img: {cvt_to_list} 
    shape img: {cvt_to_list.shape}
    dtype img: {cvt_to_list.dtype}'''
    )
    
    print(f"{'Convert to Numeric'.center(61, '=')}")
    cvt_to_num = image = tf.strings.to_number(cvt_to_list, out_type=tf.float32)
    print(f'''    vector of img: {cvt_to_num} 
    shape img: {cvt_to_num.shape}
    dtype img: {cvt_to_num.dtype}
    target image ndim: {tf.math.sqrt(cvt_to_num.shape[0] / 1.)}
    max intensity: {tf.reduce_max(cvt_to_num)}
    min intensity: {tf.reduce_min(cvt_to_num)}'''
    )

    print(f"{'Reshape to height and width dimension'.center(61, '=')}")
    cvt_to_img = tf.reshape(cvt_to_num, (96, 96))
    print(f'''    matriks of img: {cvt_to_img} 
    shape img: {cvt_to_img.shape}
    dtype img: {cvt_to_img.dtype}
    max intensity: {tf.reduce_max(cvt_to_img)}
    min intensity: {tf.reduce_min(cvt_to_img)}'''
    )

    print(f"{'Normalized image'.center(61, '=')}")
    normalized_img = (cvt_to_img / 255) * 1.0
    print(f'''    matriks of img: {normalized_img} 
    shape img: {normalized_img.shape}
    dtype img: {normalized_img.dtype}
    max intensity: {tf.reduce_max(normalized_img)}
    min intensity: {tf.reduce_min(normalized_img)}'''
    )

    for i in range(0, len(keypoint), 2):
        x = keypoint[i]
        y = keypoint[i+1]
        plt.scatter(x, y, s=75, marker='.', c='blue')
    plt.imshow(normalized_img, cmap='gray')
plt.show()

In [ ]:
def preprocessing_train_data(image, keypoint=None, input_shape=None):
    image = tf.strings.split(image, sep=' ')
    image = tf.strings.to_number(image, out_type=tf.float32)
    image = (image / 255) * 1.0
    image = tf.reshape(image, input_shape)
    return image, keypoint

def replace_nan_with_zero(image, keypoint):
    keypoint = tf.where(tf.math.is_nan(keypoint), tf.zeros_like(keypoint), keypoint)
    return image, keypoint

def get_nan_in_data(image, keypoint):
    # Memeriksa apakah ada NaN dalam keypoint
    get_nan_in_data = tf.reduce_any(tf.math.is_nan(keypoint))
    return get_nan_in_data

In [ ]:
get_nan_infacial_keypoints_train_datasets = facial_keypoints_train_datasets.filter(get_nan_in_data)

print(get_nan_infacial_keypoints_train_datasets)
for image, keypoint in get_nan_infacial_keypoints_train_datasets.take(1):
    print("Image shape:", image.shape)
    print("Keypoints:", keypoint.numpy())

In [ ]:
canny = Canny(sigma=0.9, threshold_min=50, threshold_max=100, tracking_con=5, tracking_iterations=8)

facial_keypoints_train_datasets_processed = facial_keypoints_train_datasets.map(
    map_func=lambda image, keypoint: 
        preprocessing_train_data(
            image=image, 
            keypoint=keypoint,
            input_shape=(96, 96, 1)
        ),
    num_parallel_calls=tf.data.AUTOTUNE
)

facial_keypoints_train_datasets_flip_h = facial_keypoints_train_datasets_processed.map(
    map_func=lambda image, keypoint: 
        flip_coordinate_and_image_horizontal_in_tf_dataset(
            image=image,
            keypoint=keypoint
        ),
    num_parallel_calls=tf.data.AUTOTUNE
)

facial_keypoints_concatenated_train_dataset = facial_keypoints_train_datasets_processed.concatenate(facial_keypoints_train_datasets_flip_h)

facial_keypoints_train_datasets_not_nan = facial_keypoints_concatenated_train_dataset.map(
    map_func=lambda image, keypoint: 
        replace_nan_with_zero(
            image=image, 
            keypoint=keypoint,
        ),
    num_parallel_calls=tf.data.AUTOTUNE
)

facial_keypoints_concatenated_train_dataset_cached = facial_keypoints_train_datasets_not_nan.cache()

In [ ]:
print(f'info data: {facial_keypoints_concatenated_train_dataset_cached}')
print(f'number of data: {len(facial_keypoints_concatenated_train_dataset_cached)}')

In [ ]:
for img, keypoint in facial_keypoints_concatenated_train_dataset_cached.skip(80).take(1).as_numpy_iterator():
    print(f"{'Check data'.center(61, '=')}")
    print(f'''    shape img: {img.shape}
    dtype img: {img.dtype}
    max intensity: {tf.reduce_max(img)}
    min intensity: {tf.reduce_min(img)}
    keypoint value: {keypoint}
    dtype keypoint: {keypoint.dtype}'''
    )

    for i in range(0, len(keypoint), 2):
        x = keypoint[i]
        y = keypoint[i+1]
        plt.scatter(x, y, s=75, marker='.', c='blue')
    plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
get_nan_infacial_keypoints_train_datasets = facial_keypoints_concatenated_train_dataset_cached.filter(get_nan_in_data)

print(get_nan_infacial_keypoints_train_datasets)
for image, keypoint in get_nan_infacial_keypoints_train_datasets.take(1):
    print("Image shape:", image.shape)
    print("Keypoints:", keypoint.numpy())

In [ ]:
print('Original Datasets')
tf.random.set_seed(12)
show_multiple_images_with_keypoints_in_tf_datasets(
    dataset=facial_keypoints_concatenated_train_dataset_cached.take(int(len(facial_keypoints_concatenated_train_dataset_cached)/2)), 
    num_images=12, 
    figsize_per_image=(2, 2), 
    scatter_size=10, 
    scatter_color='blue'
)

In [ ]:
print('Fliped Datasets')
tf.random.set_seed(12)
show_multiple_images_with_keypoints_in_tf_datasets(
    dataset=facial_keypoints_concatenated_train_dataset_cached.skip(int(len(facial_keypoints_concatenated_train_dataset_cached)/2)), 
    num_images=12, 
    figsize_per_image=(2, 2), 
    scatter_size=10, 
    scatter_color='blue'
)

In [ ]:
# facial_keypoints_concatenated_train_dataset_cached.save("./Assets/datasets/facial_keypoints_train_dataset.tfrecord", compression="GZIP")

In [ ]:
test_data_copy = test_data.copy(deep=True)

In [ ]:
facial_keypoints_test_datasets = tf.data.Dataset.from_tensor_slices(test_data_copy['Image'].values)

print(f'info data: {facial_keypoints_test_datasets}')
print(f'number of data: {len(facial_keypoints_test_datasets)}')

In [ ]:
for img in facial_keypoints_test_datasets.take(1).as_numpy_iterator():
    print(f"{'Check data'.center(61, '=')}")
    print(f'''    img value: {img[:20]}
    dtype img: {type(img)}'''
    )

    print(f"{'Convert to List'.center(61, '=')}")
    cvt_to_list = tf.strings.split(img, sep=' ')
    print(f'''    vector of img: {cvt_to_list} 
    shape img: {cvt_to_list.shape}
    dtype img: {cvt_to_list.dtype}'''
    )
    
    print(f"{'Convert to Numeric'.center(61, '=')}")
    cvt_to_num = image = tf.strings.to_number(cvt_to_list, out_type=tf.float32)
    print(f'''    vector of img: {cvt_to_num} 
    shape img: {cvt_to_num.shape}
    dtype img: {cvt_to_num.dtype}
    target image ndim: {tf.math.sqrt(cvt_to_num.shape[0] / 1.)}
    max intensity: {tf.reduce_max(cvt_to_num)}
    min intensity: {tf.reduce_min(cvt_to_num)}'''
    )

    print(f"{'Reshape to height and width dimension'.center(61, '=')}")
    cvt_to_img = tf.reshape(cvt_to_num, (96, 96))
    print(f'''    matriks of img: {cvt_to_img} 
    shape img: {cvt_to_img.shape}
    dtype img: {cvt_to_img.dtype}
    max intensity: {tf.reduce_max(cvt_to_img)}
    min intensity: {tf.reduce_min(cvt_to_img)}'''
    )

    print(f"{'Normalized image'.center(61, '=')}")
    normalized_img = (cvt_to_img / 255) * 1.0
    print(f'''    matriks of img: {normalized_img} 
    shape img: {normalized_img.shape}
    dtype img: {normalized_img.dtype}
    max intensity: {tf.reduce_max(normalized_img)}
    min intensity: {tf.reduce_min(normalized_img)}'''
    )

    for i in range(0, len(keypoint), 2):
        x = keypoint[i]
        y = keypoint[i+1]
        plt.scatter(x, y, s=75, marker='.', c='blue')
    plt.imshow(normalized_img, cmap='gray')
plt.show()

In [ ]:
def preprocessing_test_data(image, input_shape=None):
    # Split the image string into a list of numbers
    image = tf.strings.split(image, sep=' ')
    # Convert the split strings into numbers
    image = tf.strings.to_number(image, out_type=tf.float32)
    image = (image / 255) * 1.0
    # Reshape the image to its original shape (e.g., 96x96 if the image is 96x96 pixels)
    image = tf.reshape(image, input_shape)
    return image

In [ ]:
facial_keypoints_test_datasets_processed = facial_keypoints_test_datasets.map(
    map_func=lambda image: 
        preprocessing_test_data(
            image=image, 
            input_shape=(96, 96, 1)
        ),
    num_parallel_calls=tf.data.AUTOTUNE
)

facial_keypoints_test_dataset_cached = facial_keypoints_test_datasets_processed.cache()

In [ ]:
print(f'info data: {facial_keypoints_test_dataset_cached}')
print(f'number of data: {len(facial_keypoints_test_dataset_cached)}')

In [ ]:
for img in facial_keypoints_test_dataset_cached.skip(1).take(1).as_numpy_iterator():
    print(f"{'Check data'.center(61, '=')}")
    print(f'''    shape img: {img.shape}
    dtype img: {img.dtype}
    max intensity: {tf.reduce_max(img)}
    min intensity: {tf.reduce_min(img)}
    keypoint value: {keypoint}
    dtype keypoint: {keypoint.dtype}'''
    )
    plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
show_multiple_images_with_keypoints_in_tf_datasets(facial_keypoints_test_dataset_cached.map(lambda x: (x, None)), num_images=6)

In [ ]:
facial_keypoints_test_dataset_cached.save("./Assets/datasets/facial_keypoints_test_dataset.tfrecord", compression="GZIP")